In [1]:
import base64
import mysql.connector as mysql
import datetime
import pandas as pd
import time

#######
## INIT
#######

# load api key
secret = {}
with open('secret.txt') as f:
    lines = f.readlines()
    for line in lines:
        secret[line.split("=")[0]] = line.split("=")[1].replace("\n","")

# Load blacklist

In [2]:
blacklist = pd.read_csv('blacklist.csv', header=None, names=['handle'])
blacklist.drop_duplicates(inplace=True)
blacklist.shape

(70, 1)

In [3]:
blacklist = blacklist['handle'].tolist()

# Remove blacklist users from twitter and rewards tables

In [4]:
for i, handle in enumerate(blacklist):
    
    print(f"Removing blacklist user {i+1}/{len(blacklist)} {handle}")

    # delete from twitter
    db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
    cursor = db.cursor()
    query = "DELETE FROM twitter WHERE twitter_handle = %s;"
    values = (handle,)
    cursor.execute(query, values)
    db.commit()
    print(cursor.rowcount, "record updated")
    cursor.close()
    db.close()  

    # delete from rewards
    db = mysql.connect(host=secret['DBHOST'],user=secret['DBUSER'],passwd=secret['DBPASS'],database=secret['DBTABLE'])
    cursor = db.cursor()
    query = "DELETE FROM rewards WHERE twitter_handle = %s;"
    values = (handle,)
    cursor.execute(query, values)
    db.commit()
    print(cursor.rowcount, "record updated")
    cursor.close()
    db.close()  
    

Removing blacklist user 1/70 tabah_rafli
3 record updated
2 record updated
Removing blacklist user 2/70 gregjoh71430076
1 record updated
1 record updated
Removing blacklist user 3/70 ratucondong
5 record updated
4 record updated
Removing blacklist user 4/70 treblex11
1 record updated
1 record updated
Removing blacklist user 5/70 frankpullman9
1 record updated
1 record updated
Removing blacklist user 6/70 daanqi671
1 record updated
1 record updated
Removing blacklist user 7/70 basemycamp
1 record updated
1 record updated
Removing blacklist user 8/70 arkus_eth
2 record updated
2 record updated
Removing blacklist user 9/70 hoaido0205
1 record updated
1 record updated
Removing blacklist user 10/70 blues20cha
1 record updated
1 record updated
Removing blacklist user 11/70 linlin_big
1 record updated
1 record updated
Removing blacklist user 12/70 luffy__0606
3 record updated
2 record updated
Removing blacklist user 13/70 ji_zuma
1 record updated
1 record updated
Removing blacklist user 14/70

KeyboardInterrupt: 